In [1]:
import json
import csv
import pandas as pd
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
from nltk.sentiment import SentimentIntensityAnalyzer


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Error loading punkt: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>
[nltk_data] Error loading stopwords: <urlopen error [WinError 10054]
[nltk_data]     An existing connection was forcibly closed by the
[nltk_data]     remote host>
[nltk_data] Error loading wordnet: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>


False

In [2]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\qfu88\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

### Preprocessing the data

In [3]:
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # remove special characters, punctuation, and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # tokenize the text
    tokens = word_tokenize(text)
    # lemmatize and remove stopwords
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stopwords.words('english')]
    # join the tokens back into a string
    return ' '.join(lemmatized_tokens)


In [4]:
df = pd.read_csv('Job_Datasets/final_data.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8758 entries, 0 to 8757
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      7897 non-null   float64
 1   text                    8758 non-null   object 
 2   title                   8758 non-null   object 
 3   author                  8648 non-null   object 
 4   num_comments            8758 non-null   int64  
 5   post_id                 8758 non-null   object 
 6   upvote_ratio            8758 non-null   float64
 7   score                   8758 non-null   int64  
 8   url                     8758 non-null   object 
 9   subreddit               8758 non-null   object 
 10  link_flair_text         8758 non-null   object 
 11  link_flair_template_id  8758 non-null   object 
 12  created_datetime        8758 non-null   object 
 13  day_of_week             8758 non-null   object 
 14  hour_of_day             8758 non-null   

In [6]:
# Dropping the 'id' column
df = df.drop(columns=['id'])

# Dropping rows with missing values in the 'author' column
df = df.dropna(subset=['author'])


In [7]:
df['text'] = df['text'].apply(preprocess_text)

In [8]:
df['text'].value_counts()

text
xBnn                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 46
Title      

### further cleaning 

In [9]:
df = df[~df['text'].str.contains('xBnn|Title|^Please use thread|^ReadOnly Friday', case=False, na=False)]

In [10]:
df.shape

(7852, 16)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7852 entries, 0 to 8757
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   text                    7852 non-null   object 
 1   title                   7852 non-null   object 
 2   author                  7852 non-null   object 
 3   num_comments            7852 non-null   int64  
 4   post_id                 7852 non-null   object 
 5   upvote_ratio            7852 non-null   float64
 6   score                   7852 non-null   int64  
 7   url                     7852 non-null   object 
 8   subreddit               7852 non-null   object 
 9   link_flair_text         7852 non-null   object 
 10  link_flair_template_id  7852 non-null   object 
 11  created_datetime        7852 non-null   object 
 12  day_of_week             7852 non-null   object 
 13  hour_of_day             7852 non-null   int64  
 14  month                   7852 non-null   objec

In [13]:
df.drop_duplicates(subset='text', keep='first', inplace=True)


In [14]:
df.shape

(7761, 16)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7761 entries, 0 to 8757
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   text                    7761 non-null   object 
 1   title                   7761 non-null   object 
 2   author                  7761 non-null   object 
 3   num_comments            7761 non-null   int64  
 4   post_id                 7761 non-null   object 
 5   upvote_ratio            7761 non-null   float64
 6   score                   7761 non-null   int64  
 7   url                     7761 non-null   object 
 8   subreddit               7761 non-null   object 
 9   link_flair_text         7761 non-null   object 
 10  link_flair_template_id  7761 non-null   object 
 11  created_datetime        7761 non-null   object 
 12  day_of_week             7761 non-null   object 
 13  hour_of_day             7761 non-null   int64  
 14  month                   7761 non-null   objec

In [17]:
df.to_csv('Sentiment & Engagement Datasets/ready_data.csv', index=False)

### Performing Sentiment Analysis with VADER